# Dataset creation

## Installation

In [4]:
!pip install -q llama-cpp-python textdescriptives argilla transformers datasets langdetect


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!python -m spacy download en_core_web_sm

In [3]:
!CT_METAL=1 pip install -q ctransformers --no-binary ctransformers

DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# from ctransformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral")

# print(llm("AI is going to"))

## Imports

In [28]:
import pandas as pd
import textdescriptives as td
from datasets import load_dataset
import re
import spacy
from langdetect import detect
import argilla as rg

## Pre-processing

End-to-end workflow to create a dataset in Argilla with text measurements as metadata.
This aids in quickly identifying and improving potential dataset issues.

### Dataset creation

At first, we need to create a dataset in Argilla. This can either be done by loading a previous created dataset or by creating a new one. In order to avoid duplication, we will check if the dataset already exists.

In [26]:
try:
    ds_local = rg.FeedbackDataset.for_supervised_fine_tuning(context=True, use_markdown=True, guidelines=None)
    ds_local.questions.extend([
        rg.RatingQuestion(
            name="prompt-quality", 
            title="Prompt Quality",
            values=list(range(1, 8)), 
            description="How would you rate the quality of the prompt?",
        ),
        rg.LabelQuestion(
            name="prompt-intent", 
            title="Prompt Intent",
            labels=["generation", "rewrite", "extract", "closed-qa", "open-qa", "classification", "summarization", "brainstorming", "chat", "code", "other"], 
            description="What is the intent of the prompt?"
        ),
        rg.MultiLabelQuestion(
            name="prompt-toxicity", 
            title="Prompt Toxicity",
            labels=["illegal", "harmfull", "unqualified advice"], 
            description="What are the toxicities in the prompt (if any)?",
            required=False
        )
    ])
    ds_remote = ds_local.push_to_argilla("sharegpt")
except Exception as e:
    ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")
ds_remote

RemoteFeedbackDataset(
   id=8eb16dbb-1019-40d9-aa8d-b6b25164a268
   name=sharegpt
   workspace=Workspace(id=e41766d9-89eb-4497-afce-1d96c7ff5f51, name=argilla, inserted_at=2023-10-28 11:26:52.619081, updated_at=2023-10-28 11:26:52.619081)
   url=http://localhost:6900/dataset/8eb16dbb-1019-40d9-aa8d-b6b25164a268/annotation-mode
   fields=[RemoteTextField(id=UUID('9614aac2-c840-4715-9095-d751cb0ca79e'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('680f6051-5c48-46fe-bd14-8253c23660d6'), client=None, name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('0214cd83-42f6-4a42-825b-2c986adbd4e8'), client=None, name='response', title='Response', description=None, required=True, type='text', use_markdown=True), RemoteRatingQuestion(id=UUID('9ede196e-6e3f-43f0-8d81-59fe1de05399'), client=None, name='prompt-quality', title='Prompt Quality', description=None

#### Configure the metadata-properties

Next we will be using `text-descriptives` to configure the metadata-properties. This will be used to add and updat relevant metadata-properties to the dataset. Because `text-descriptives` doesn't provide any programmatic interface with the metrics-groups and their sub-metrics, we will run the computation on the an example text and use the results to configure the metadata-properties.

In [37]:
metric_group = ["descriptive_stats"]
relevant_subgroups = []
df_metrics = td.extract_metrics(
    text=["this is an example prompt"], 
    lang="en", 
    metrics=metric_group,
    spacy_model="en_core_web_sm"
).drop(columns=["text"] + relevant_subgroups if relevant_subgroups else ["text"])
df_metrics.columns

ℹ Both a spacy model and a language were provided. Will use the spacy
model and ignore language.


Index(['token_length_mean', 'token_length_median', 'token_length_std',
       'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
       'syllables_per_token_mean', 'syllables_per_token_median',
       'syllables_per_token_std', 'n_tokens', 'n_unique_tokens',
       'proportion_unique_tokens', 'n_characters', 'n_sentences'],
      dtype='object')

Next, we will be working on converting the `text-descriptives` output to a format that can be used to configure the metadata-properties for our supported types: `TermsMetadataProperty`, `IntegerMetadataProperty` and `FloatMetadataProperty`. Note that we are also applying some subjective formatting choices to ensure that the metadata-properties are easy to read and understand.

In [38]:
def clean_column_name(col_name):
    """Clean a column name to fit a specific regex pattern."""
    col_name = col_name.lower()  # Convert to lowercase
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Replace non-alphanumeric characters with underscores
    return col_name

def create_metadata_properties(df, prefix):
    """Generate metadata properties based on dataframe columns and data types."""
    properties = []
    for col, dtype in df.dtypes.items():
        name = f"{prefix}_{clean_column_name(col)}"
        title = name.replace('_', ' ').title()

        if dtype == 'object':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        elif dtype == 'int64':
            prop = rg.IntegerMetadataProperty(name=name, title=title)
        elif dtype == 'float64':
            prop = rg.FloatMetadataProperty(name=name, title=title)
        elif dtype == 'bool':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        else:
            print(f"Unhandled data type for column {col}: {dtype}")
            continue
        properties.append(prop)
    return properties

metadata_properties = create_metadata_properties(df_metrics, 'prompt')
for metadata_property in metadata_properties:
    try:
        ds_remote.metadata_property_by_name(metadata_property.name)
    except KeyError:
        ds_remote.add_metadata_property(metadata_property)

### Data collection

In [ ]:
dataset = load_dataset("totally-not-an-llm/sharegpt-hyperfiltered-3k", split="train")
dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")
dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x["conversations"][1]["value"]})

# Extract metrics
df_prompt = td.extract_metrics(text=dataset["prompt"], lang="en", spacy_model="en").drop(columns=['text'])
df_response = td.extract_metrics(text=dataset["response"], lang="en", spacy_model="en").drop(columns=['text'])

# Identify integer and boolean columns for prompts and responses
int_cols_prompts = df_prompt.select_dtypes(include=['int64']).columns.tolist()
bool_cols_prompts = df_prompt.select_dtypes(include=['boolean']).columns.tolist()

int_cols_responses = df_response.select_dtypes(include=['int64']).columns.tolist()
bool_cols_responses = df_response.select_dtypes(include=['boolean']).columns.tolist()

# Combine column lists for prompts and responses
int_cols = list(set(int_cols_prompts + int_cols_responses))
bool_cols = list(set(bool_cols_prompts + bool_cols_responses))

###

In [5]:
"""

"""


# --- Functions ---

def cast_to_python_types(df):
    """
    Convert integer and boolean columns to Python native types.
    """
    int_cols = df.select_dtypes(include=['int64']).columns
    bool_cols = df.select_dtypes(include=['boolean']).columns

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        df[col] = df[col].apply(int)

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        df[col] = df[col].apply(str)

    return df

def detect_language(text):
    """
    Detect the language of a given text.

    Args:
    - text (str): Input text.

    Returns:
    - str: Detected language (ISO 639-1 code).
    """
    try:
        return detect(text)
    except:
        return "unknown"  # In case the language detection fails




# --- Metadata Preparation ---
metadata_prompt = create_metadata_properties(df_prompt, 'prompt')
metadata_response = create_metadata_properties(df_response, 'response')

all_metadata = metadata_prompt + metadata_response

ds = rg.FeedbackDataset.for_supervised_fine_tuning(context=True, use_markdown=True, guidelines=None)
for m in all_metadata:
    ds.add_metadata_property(m)

# --- Record Preparation ---
records = []

# Prepare feedback records with metadata and suggestions

# Identify columns with values other than zeros or NaN for both prompt and response
cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all()].tolist()


records = []

cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all() & ~df_prompt.isnull().any()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all() & ~df_response.isnull().any()].tolist()

ds = rg.FeedbackDataset.for_supervised_fine_tuning(context=True, use_markdown=True, guidelines=None)
for m in all_metadata:
    ds.add_metadata_property(m)

for i, record in enumerate(dataset):
    # Prepare metadata for prompts
    metadata_prompts = {f"prompt_{col}": value for col, value in df_prompt[cols_with_values_other_than_zeros_or_nan_prompt].iloc[i].items()}
    # Prepare metadata for responses
    metadata_response = {f"response_{col}": value for col, value in df_response[cols_with_values_other_than_zeros_or_nan_response].iloc[i].items()}
    if "prompt_smog" in metadata_prompts.keys():
      print(metadata_prompts)

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = int(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = int(metadata_response[f"response_{col}"])

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = str(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = str(metadata_response[f"response_{col}"])

    # Combine both metadata dictionaries into one
    metadata = {**metadata_prompts, **metadata_response}

    records.append(
        rg.FeedbackRecord(
            fields={"prompt": record["prompt"]},
            metadata=metadata,
            suggestions=[{"question_name": "response", "value": record["response"]}]
        )
    )

# Add records to the dataset and push to Argilla
ds.add_records(records)
ds.push_to_argilla(name="share-gpt-descriptives")

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 460.00it/s]
Generating train split: 3243 examples [00:00, 68097.46 examples/s]
Map: 100%|██████████| 3241/3241 [00:00<00:00, 11316.58 examples/s]


ℹ No spacy model provided. Inferring spacy model for en.
Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:02



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/textdescriptives/components/coherence.py:44: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  similarities.append(sent.similarity(sents[i + order]))


ℹ No spacy model provided. Inferring spacy model for en.
Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.0/en_core_web_lg-3.7.0-py3-none-any.whl (587.7 MB)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/textdescriptives/components/coherence.py:44: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  similarities.append(sent.similarity(sents[i + order]))
/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/argilla/client/client.py:154: UserWarning: Default user was detected and no workspace configuration was provided, so the default 'argilla' workspace will be used. If you want to setup another workspace, use the `rg.set_workspace` function or provide a different one on `rg.init`
  warnings.warn(
Pushing records to Argilla...: 100%|██████████| 102/102 [00:20<00:00,  5.04it/s]


RemoteFeedbackDataset(
   id=853fc94e-6e92-4361-927c-3100024394ad
   name=share-gpt-descriptives
   workspace=Workspace(id=e41766d9-89eb-4497-afce-1d96c7ff5f51, name=argilla, inserted_at=2023-10-28 11:26:52.619081, updated_at=2023-10-28 11:26:52.619081)
   url=http://localhost:6900/dataset/853fc94e-6e92-4361-927c-3100024394ad/annotation-mode
   fields=[RemoteTextField(id=UUID('0c4d73f7-83f7-4501-86ce-6a06ceb075e3'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('07c28c08-51d6-40d6-9bb1-ebdd33b7348c'), client=None, name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('cfe300a8-3ff8-4284-b14f-0e3175b055d2'), client=None, name='response', title='Response', description=None, required=True, type='text', use_markdown=True)]
   guidelines=This is a supervised fine-tuning dataset that contains instructions. Please write the response to the instruction in t

## Questions

You can reuse the dataset https://huggingface.co/datasets/argilla/sharegpt-text-descriptives